In [ ]:
# %run ./refresh_token

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# **Connection**

In [ ]:
pip install praw

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 188 kB 21.9 MB/s 
     |████████████████████████████████| 54 kB 2.6 MB/s 


In [ ]:
import json
from datetime import datetime

credentials = 'client_secret.json'

with open(credentials) as f:
    creds = json.load(f)

In [ ]:
import praw
reddit = praw.Reddit(client_id = creds['client_id'],
                    client_secret = creds['client_secret'],
                    user_agent = creds['user_agent'],
                    redirect_uri = creds['redirect_uri'],
                    refresh_token = creds['refresh_token'],
                     check_for_async = False)

# **Loading removed comments**

In [ ]:
import pandas as pd

science_removed_com_df = pd.read_csv('/content/non_single_high_tox_science_10to24may.csv')
# science_tox_removed_com_df = pd.read_csv('/content/toxicity_removed_com_science_12to19apr.csv')
# rance_removed_com_df = pd.read_csv('/content/removed_com_rance_6to12apr.csv')
# france_removed_com_df = pd.read_csv('/content/removed_com_france_9to15may.csv')
# europe_removed_com_df = pd.read_csv('/content/removed_com_europe_9to15may.csv')
# askeurope_removed_com_df = pd.read_csv('/content/removed_com_askeurope_6to12apr.csv')
# geop_removed_com_df = pd.read_csv('/content/removed_com_geopolitics_6to12apr.csv')

In [ ]:
# datetime.fromtimestamp(removed_com_df.created_utc.min())

In [ ]:
# find_com = reddit.comment(id = 'hzlpqbr')

In [ ]:
# find_com.parent_id

In [ ]:
def find_org_body(id, df):
  com = df[df.id == id]
  if len(com) == 0:
    return ["NOT FOUND"]
  # return com.values[0][0]
  return ["FOUND", com.values[0][1], com.values[0][0]]

In [ ]:
t = find_org_body('i7yzgus', science_removed_com_df)

In [ ]:
print(t)

['FOUND', 'J’ai 21 ans, et non elle a avorter.', 'mwijuo7n']


In [ ]:
print(t[1])

J’ai 21 ans, et non elle a avorter.


In [ ]:
def find_post(id):
  post = reddit.submission(id = str(id))
  return post

In [ ]:
def find_parent(com, level, parent_dict):
  result = parent_dict

  if com.parent_id == com.link_id:
    try:
      parent_post = find_post(com.parent_id[3:])

      sub_dict = {}
      sub_dict['id'] = parent_post.id
      sub_dict['title'] = parent_post.title
      sub_dict['selftext'] = parent_post.selftext
      sub_dict['score'] = parent_post.score
      sub_dict['upvote_ratio'] = parent_post.upvote_ratio
      sub_dict['num_comments'] = parent_post.num_comments
      sub_dict['permalink'] = parent_post.permalink
      sub_dict['created_utc'] = parent_post.created_utc

      try:
        if parent_post.author != None:
          sub_dict['author'] = parent_post.author.id
        else:
          sub_dict['author'] = 'Not found'
      except Exception as e:
        print("Post Author ERROR: author is ", parent_post.author, "Exception: ", e)
        sub_dict['author'] = 'Not found'

      result[level] = ["Level "+str(level), "Parent Post", sub_dict]
      return result

    except Exception as e:
      print("ERROR: ", e)
      print("Comment number:" + com.id)
      
  else:
    try:
      p = reddit.comment(id = com.parent_id[3:])

      com_dict = {}
      com_dict['id'] = p.id
      com_dict['parent_id'] = p.parent_id
      com_dict['link_id'] = p.link_id
      com_dict['body'] = p.body
      com_dict['collapsed'] = 0 if p.collapsed == False else 1
      com_dict['score'] = p.score
      com_dict['controversiality'] = p.controversiality
      com_dict['permalink'] = p.permalink
      com_dict['created_utc'] = p.created_utc

      try: 
        if p.author != None:
          com_dict ['author'] = p.author.id
        else:
          com_dict ['author'] = 'Not found'
      except Exception as e:
        com_dict ['author'] = 'Not found'
        print("Comment Author ERROR: author is ", p.author, "Exception: ", e)

      if p.body == '[removed]':
        f = find_org_body(p.id, science_removed_com_df)
        if f[0] == 'NOT FOUND':
          com_dict['body'] =  '[removed] and NOT FOUND'
        else:
          com_dict['body'] = '[removed] '+ f[1]
          com_dict ['author'] = f[2]

        result[level] = ["Level "+str(level), "Parent comment [removed]", com_dict]
      else:
        result[level] = ["Level "+str(level), "Parent comment", com_dict]

      level = level +1
      find_parent(p, level, result)

    except Exception as e:
      print("ERROR: ", e)
      print("Comment number:" + com.id)

  return result

In [ ]:
result_json = {}
temp_list = []

counter = 1

for idx, com in science_removed_com_df.iterrows():
  print("-----count: ", counter)
  temp = {}
  x = find_parent(com, 1, temp)
  
  com_dict = com.to_dict()
  temp = com_dict['body']
  com_dict['body'] = '[removed] ' + temp
  final_thread = {0: ["Level 0", "Removed comment", com_dict], **x}
  
  temp_list.append(final_thread)
  print("RESULT: ", final_thread)
  counter +=1

result_json['result'] = temp_list

with open("non_single_tox_science_discussions_10to24may.json", "w") as outfile:
    json.dump(result_json, outfile)

In [ ]:
len(temp_list)

285

In [ ]:
result_json['result'] = temp_list

with open("first_285_non_single_tox_science_discussions_10to24may.json", "w") as outfile:
    json.dump(result_json, outfile)

In [ ]:
from google.colab import files
files.download("/content/first_285_non_single_tox_science_discussions_10to24may.json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
filtered_list = [thread for thread in temp_list if len(thread.keys()) >= 5]

In [ ]:
len(filtered_list)

144

In [ ]:
import json
f = open("/content/last_200_high_tox_science_discussions_10to24may.json")
last_200_json_file = json.load(f)

In [ ]:
last_200 = last_200_json_file['result']

In [ ]:
filtered_200 = [thread for thread in last_200 if len(thread.keys()) >= 5]

In [ ]:
len(filtered_200)

117

In [ ]:
filtered_list.extend(filtered_200)

In [ ]:
len(filtered_list)

261

In [ ]:
new_res= {}
new_res['result'] = filtered_list

with open("req_science_discussions_10to24may.json", "w") as outfile:
    json.dump(new_res, outfile)

In [ ]:
files.download('/content/req_science_discussions_10to24may.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import json
# f = open("/content/science_discussions_9to15may.json")
f = open('/content/req_science_discussions_10to24may.json')
json_file = json.load(f)

In [ ]:
final = []
for discussion_dict in json_file['result']:
  discussion_dict = {int(k): v for k,v in discussion_dict.items()}
  sorted_dict = dict(sorted(discussion_dict.items(), reverse=True))
  new_dict = {}
  i = 0
  for k,v in sorted_dict.items():
    new_dict[i] = v[2]
    i+=1
  
  final.append(new_dict)

In [ ]:
final_json = {}
final_json['result'] = final

In [ ]:
with open("science_discussions_details_9to15may.json", "w") as outfile:
    json.dump(final_json, outfile)

In [ ]:
from google.colab import files
files.download("/content/science_discussions_details_9to15may.json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
new_f = open("/content/france_discussions_details_9to15may.json")
new_json_file = json.load(new_f)

In [ ]:
author_text = []

for item in final_json['result']:
  print(item)
  dis ={}
  for k,v in item.items():
    if k == 0:
      dis[k] = {"author" : v['author'], "content" : v['title'] + v['selftext']}
    else:
      dis[k] = {"author": v['author'] , "content": v['body']}
  
  author_text.append(dis)

{0: {'id': 'ulyz57', 'title': 'Young men with a poor diet saw a significant improvement in their symptoms of depression when they switched to a healthy Mediterranean diet. The diet used in the study was rich in colourful vegetables, legumes and wholegrains, oily fish, olive oil and raw, unsalted nuts.', 'selftext': '', 'score': 63454, 'upvote_ratio': 0.9, 'num_comments': 2524, 'permalink': '/r/science/comments/ulyz57/young_men_with_a_poor_diet_saw_a_significant/', 'created_utc': 1652124118.0, 'author': 'guf36'}, 1: {'id': 'i7ypna9', 'parent_id': 't3_ulyz57', 'link_id': 't3_ulyz57', 'body': 'It doesn’t help that nutritionists are basically chiropractors. Our knowledge of nutrition is really scattered and impacted by fad diets and medical-adjacent fields trying to profit on a fuzzy concept.', 'collapsed': 0, 'score': 105, 'controversiality': 0, 'permalink': '/r/science/comments/ulyz57/young_men_with_a_poor_diet_saw_a_significant/i7ypna9/', 'created_utc': 1652127882.0, 'author': 'me5vk'},

In [ ]:
author_text_json = {}
author_text_json['result'] = author_text
with open("science_discussions_author_and_text_9to15may.json", "w") as outfile:
    json.dump(author_text_json, outfile)

In [ ]:
from google.colab import files
files.download("/content/science_discussions_author_and_text_9to15may.json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **for science threads for demo**

In [ ]:
import json

req_author_text_f = open("/content/req_science_discussions_author_and_text_10to24may.json")
req_author_text = json.load(req_author_text_f)

In [ ]:
req_list = req_author_text['result']

In [ ]:
# indexs = [7,19,99,200, 260, 28, 31, 106,249]
# incomplete_threads = [req_list[i] for i in indexs]

In [ ]:
# incomplete_threads

In [ ]:
req_json = {}
req_json['result'] = incomplete_threads

with open("extra_incomplete_science_threads_10to24may.json", "w") as outfile:
    json.dump(req_json, outfile)

In [ ]:
from google.colab import files
files.download("/content/extra_incomplete_science_threads_10to24may.json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **other**

In [ ]:
only_content = []

for item in new_json_file['result']:
  print(item)
  dis ={}
  for k,v in item.items():
    if k == '0':
      dis[k] = v['title'] + v['selftext']
    else:
      dis[k] = v['body']
  
  only_content.append(dis)

In [ ]:
only_text_json = {}
only_text_json['result'] = only_content
with open("france_threads_only_text_9to15may.json", "w") as outfile:
    json.dump(only_text_json, outfile)

In [ ]:
from google.colab import files
files.download("/content/france_threads_only_text_9to15may.json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import json

only_text_f = open("/content/france_threads_only_text_9to15may.json")
only_text_threads = json.load(only_text_f)

In [ ]:
with open("all_france_threads_for_rtg.txt", "w") as outfile:
  for thread in only_text_threads['result']:
    # print(thread)
    temp = [v.replace("'", "’") for k,v in thread.items()]
    t = json.dumps({"source":temp}, ensure_ascii=False)
    content_type = "Content-Type: application/json"
    # print(f"curl -X POST -H {json.dumps(content_type)} http://spolin.isi.edu:6060/translate --data '{t}'\n")
    outfile.write(f"curl -X POST -H {json.dumps(content_type)} http://spolin.isi.edu:6060/translate --data '{t}'\n")
   


In [ ]:
# with open("france_threads_for_rtg.txt", "w") as outfile:
#   for element in final_france_threads:
#     content_type = "Content-Type: application/json"
#     outfile.write(f"curl -X POST -H {json.dumps(content_type)} http://spolin.isi.edu:6060/translate --data '{element}'\n")